In [1]:
#https://youtu.be/5ypQIUbpA7c?si=OjF6L76MTJJgSrBs
# works 

In [2]:
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from PIL import Image
import torch
import cv2
from transformers import GLPNImageProcessor, GLPNForDepthEstimation

2024-07-31 17:06:19.018969: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
image_file = "images/pointing.jpg"
video_file = "Videos/pointing.mp4"
image_file = "images/lamp2.jpg"

In [4]:
def extract_img(video_file):
    cam = cv2.VideoCapture(video_file)
    ret,frame = cam.read() # read 1st image
    cv2.imwrite("./images/tmp2.jpg", frame)
    cam.release()
    return

extract_img(video_file)

In [5]:
feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")

In [6]:
# detection
from ultralytics import YOLOWorld

def obj_detect(image):
    # Initialize a YOLO-World model
    model = YOLOWorld("yolov8s-world.pt")  # or select yolov8m/l-world.pt for different sizes
    model.set_classes(["laptop" , "lamp", "fan"])
    results = model.predict(image)

    for result in results:
        print("box-> ")
        print(result.boxes)
        print(result.probs)

    # Show results
    results[0].show()
    return results[0]
bounding=obj_detect(image_file)


image 1/1 /Users/davidtung/Downloads/Notebook/images/lamp2.jpg: 480x640 2 lamps, 182.3ms
Speed: 3.6ms preprocess, 182.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
box-> 
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1.])
conf: tensor([0.8883, 0.8610])
data: tensor([[2.5864e+03, 5.7355e+02, 3.0586e+03, 2.0432e+03, 8.8835e-01, 1.0000e+00],
        [1.5834e+03, 1.0483e+03, 1.9015e+03, 1.6049e+03, 8.6105e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (3024, 4032)
shape: torch.Size([2, 6])
xywh: tensor([[2822.5078, 1308.3508,  472.1970, 1469.6030],
        [1742.4641, 1326.6360,  318.0651,  556.6090]])
xywhn: tensor([[0.7000, 0.4327, 0.1171, 0.4860],
        [0.4322, 0.4387, 0.0789, 0.1841]])
xyxy: tensor([[2586.4094,  573.5493, 3058.6064, 2043.1523],
        [1583.4315, 1048.3314, 1901.4966, 1604.9404]])
xyxyn: tensor([[0.6415, 0.1897, 0.7586, 0.6756],
        [0.3927, 0.3467, 0.4716, 0.5307]])
None


In [7]:
image = Image.open(image_file)#.convert("RGB")
new_height = 480 if image.height > 480 else image.height
new_height -= (new_height %32)
new_width = int(new_height * image.width / image.height)
diff = new_height % 32

new_width = new_width - diff if diff < 16 else new_width +32 - diff
new_size = (new_width,new_height)
image = image.resize (new_size)

In [8]:
inputs = feature_extractor(images=image, return_tensors="pt")
with torch.no_grad():
    outputs = model (**inputs)
    predicted_depth = outputs.predicted_depth
    

In [9]:
pad = 16
#pad = (image.width - predicted_depth.shape[0]) // 2  # p for padding
print(pad)
output = predicted_depth.squeeze().cpu().numpy() * 1000.0
output = output[pad:-pad, pad:-pad]
image = image.crop((pad, pad, image.width - pad, image.height - pad ))

16


In [10]:
import numpy as np

def depth_to_point_cloud(depth_map, scale_factor=0.001):
    h, w = depth_map.shape
    i, j = np.meshgrid(np.arange(w), np.arange(h), indexing='xy')
    z = depth_map * scale_factor
    x = (i - w / 2) * z / w
    y = (j - h / 2) * z / h
    z = -z  # Invert the y-axis
    y = -y  # Invert the y-axis
    points = np.stack((x, y, z), axis=-1).reshape(-1, 3)
    return points

point_cloud = depth_to_point_cloud(output)
print(point_cloud)


[[    -1.2989      1.2989     -2.5977]
 [    -1.2952      1.2995      -2.599]
 [    -1.2925       1.301      -2.602]
 ...
 [      1.089     -1.0949     -2.1996]
 [     1.0916     -1.0939     -2.1976]
 [     1.0941     -1.0928     -2.1954]]


In [11]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_point_cloud(points):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=0.1)  # Plot the points
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()


In [12]:
fig, ax = plt.subplots(1,2)
ax[0].imshow(image)
#ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
ax[1].imshow(output, cmap='plasma')

plt.tight_layout()
plt.pause(5)

In [13]:
#

In [14]:
import numpy as np
import open3d as o3d

In [15]:
# Define transformation matrices
rotation_y_180 = np.array([
    [-1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]
])

rotation_z_180 = np.array([
    [-1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

In [16]:
def apply_transformation2(points, transformation_matrix):
    ones = np.ones((points.shape[0], 1))
    points_homogeneous = np.hstack((points, ones))  # Convert to homogeneous coordinates
    transformed_points_homogeneous = points_homogeneous @ transformation_matrix.T
    transformed_points = transformed_points_homogeneous[:, :3]  # Convert back to 3D coordinates
    return transformed_points

In [17]:
def apply_transformation(point_cloud, transformation_matrix):
    point_cloud.transform(transformation_matrix)
    return point_cloud

In [18]:
def extract_3d_coordinates(point_cloud):
    points = np.asarray(point_cloud.points)
    return points

In [19]:
width, height = image.size
depth_image = (output * 255 / np.max(output)).astype('uint8')
image = np.array(image)

In [20]:
def show_o3d_image(o3d_image):
    np_image = np.asarray(o3d_image)

    # Display the image using Matplotlib
    plt.imshow(np_image)
    plt.axis('off')  # Turn off axis labels
    plt.show()

In [21]:
depth_03d = o3d.geometry.Image(depth_image)
image_03d = o3d.geometry.Image(image)
#show_o3d_image(depth_image)
#show_o3d_image(image)
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(image_03d,depth_03d, convert_rgb_to_intensity=False)

In [22]:
camera_intrinsic = o3d.camera.PinholeCameraIntrinsic()
camera_intrinsic.set_intrinsics(width, height, 500, 500, width/2, height/2)


In [23]:
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, camera_intrinsic)

In [24]:
# Apply transformations
pcd = apply_transformation(pcd, rotation_y_180)
pcd = apply_transformation(pcd, rotation_z_180)


In [25]:

#print(np.asarray((pcd.points)))
o3d.visualization.draw_geometries([pcd])



In [26]:
points = extract_3d_coordinates(pcd)
print(points)

[[-0.00041487  0.00030569 -0.00068235]
 [-0.00041351  0.00030569 -0.00068235]
 [-0.00041214  0.00030569 -0.00068235]
 ...
 [ 0.00034704 -0.00025711 -0.00057647]
 [ 0.00034819 -0.00025711 -0.00057647]
 [ 0.00034934 -0.00025711 -0.00057647]]
